# Dummy Release Report
# 2019.06 Release Codename


## Table of Contents
* [Lava Test Results](#lava_test_results)
* [Second Item](#second_item)
* [Third Item](#third_item)
* [Fourth Item](#fourth_item)
* [Fifth Item](#fifth_item)
* [Sixth Item](#sixth_item)

## Sample Markdown

## Executive Summary	 	 	 	
your exective summary goes here

## Hardware Platforms
Hardware platforms goes here:
* HW1: 
* HW2: 

# Release Highlights
## Baseline versions
Release highlights goes here:
* first
* second


#### Lava Test Results
<a id="lava_test_results"></a>

In [21]:
import requests   
import json
from json2html import *
from bs4 import BeautifulSoup as bs
from IPython.display import display, HTML 
from datetime import datetime   
  
start_time = datetime.now()  

html_body = """<html>
<head>  
  <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.4.0/css/bootstrap.min.css">
  <script src="https://ajax.googleapis.com/ajax/libs/jquery/3.4.1/jquery.min.js"></script>
  <script src="https://maxcdn.bootstrapcdn.com/bootstrap/3.4.0/js/bootstrap.min.js"></script>
</head>
<body>{}</body></html>
"""

def query_rest_api(url,token=None):  
    result = ''  
    if token:  
        token_str = "token {}".format(token)  
        headers = {"Authorization": token_str}  
        result = requests.get(url, headers=headers) 
    else:  
        result = requests.get(url)   
          
    result.raise_for_status()              
    return result   
   

def make_new_coltag(soup, width1, width2): 
    col1 = soup.new_tag('col') 
    col2 = soup.new_tag('col') 
    col1['width'] = width1  
    col2['width'] = width2 
    colgrp = soup.new_tag('colgroup') 
    colgrp.append(col1) 
    colgrp.append(col2) 
    return colgrp 
     


base_url = "http://qa-reports.linaro.org" #"http://127.0.0.1:8000"
base_api_url = base_url + "/api/testruns/?" #"http://127.0.0.1:8000/api/testruns/?"   
project_filter = "build__project__slug={}"   
build_filter = "&build__version={}"    
limit="&limit=200"   
project_slug = "schneider" #"linux-stable-rc-4.14-oe" 
build_version_slug = "243" #"v4.14.74" 
filtered_url = "%s%s%s%s" % (base_api_url, project_filter, build_filter, limit)   
url = filtered_url.format(project_slug, build_version_slug)   
#print(url)  
lava_api_urlfrag = "/api/v0.1/jobs/"
  
qa_response = ''   
test_runs = []
my_token = "YOUR SQUAD TOKEN GOES HERE"
qa_response = query_rest_api(url, my_token).json()
buid_api_url = qa_response['results'][0]['build']
build_api_response = query_rest_api(buid_api_url, my_token).json()
project_api_url = build_api_response['project']
project_api_response = query_rest_api(project_api_url, my_token).json()
project_slug = project_api_response['slug']
group_api_url = project_api_response['group']
group_api_response = query_rest_api(group_api_url, my_token).json()
group_slug = query_rest_api(group_api_url, my_token).json()['slug']
testrun_abslt_url = "{}/{}/{}/build/{}".format(base_url,group_slug,project_slug,build_version_slug)

while True:            
    url = qa_response['next']   
    testruns = qa_response['results']
    testruns_cnt = 0   
    for tr in testruns:
        tests_response = query_rest_api(tr['tests'], my_token).json()
        metrics_response = query_rest_api(tr['metrics'], my_token).json()        
        #print(environment)
        if tests_response['count'] == 0 and metrics_response['count'] == 0:            
            continue 
        env_dict = {'Environment': query_rest_api(tr['environment'], my_token).json()['slug']}                              
        lavajob_api_url = tr['job_url'].rsplit('/', 3)[0] + lava_api_urlfrag + tr['job_id']
        job_description = query_rest_api(lavajob_api_url, None).json()['description']
        env_dict['TestRun'] = "<a href='{0}'>{1}</a>".format(tr['job_url'], job_description)        
        tests = tests_response['results']
        metrics = metrics_response['results']
        if tests:
            env_dict['Testsuites']= {}
            env_dict['Tests Total'] = tests_response['count']
            for test in tests:
                split_test = test['name'].split('/')   
                suite_name = split_test[0]
                test_name = split_test[1]
                suite_name = "<a href='{0}/testrun/{1}/suite/{2}/tests/'>{2}</a>".format(testrun_abslt_url,
                                                                                     tr['job_id'],suite_name)  
                if not suite_name in env_dict['Testsuites'].keys():
                    env_dict['Testsuites'][suite_name] = {}
                env_dict['Testsuites'][suite_name][test_name] = test['status']
        if metrics:
            env_dict['Metric Testsuites']= {}
            env_dict['Metrics Total'] = metrics_response['count']
            for metric in metrics:
                split_metric = metric['name'].split('/')
                suite_name = split_metric[0]
                test_name = split_metric[1]
                suite_name = "<a href='{0}/testrun/{1}/suite/{2}/metrics/'>{2}</a>".format(testrun_abslt_url,
                                                                                     tr['job_id'],suite_name)  
                if not suite_name in env_dict['Metric Testsuites'].keys():
                    env_dict['Metric Testsuites'][suite_name] = {}
                env_dict['Metric Testsuites'][suite_name][test_name] = metric['result']
                     
        test_runs.append(env_dict)        
    if url:   
        qa_response = query_rest_api(url, my_token).json()  
    else:   
        break    
#print(datetime.now()-start_time)

html_ = json2html.convert(json=json.dumps(test_runs),escape=False)
from bs4 import BeautifulSoup as bs 
 
soup = bs(html_)
soup.ul.unwrap() 
for li in soup.body.find_all('li'):  
    li.unwrap()   
cell_style = 'cellpadding: 0; border: 0.1px solid black; text-align: left; vertical-align: top;'
for td in soup.body.find_all('td'):
    td['style'] = cell_style
for th in soup.body.find_all('th'):
    th['style'] = cell_style
for tbl1 in soup.body.find_all('table', recursive=False):
    tbl1['class'] = 'table table-bordered table-sm'
    tbl1.insert(0, make_new_coltag(soup, "10%", "90%"))     
    for tbl2 in tbl1.find_all('table', limit=1): 
        tbl2['class'] = 'table table-bordered table-sm'
        tbl2.insert(0, make_new_coltag(soup, "20%", "80%")) 
        for tbl3 in tbl2.find_all('table'): 
            tbl3['class'] = 'table table-bordered table-sm'
            tbl3.insert(0, make_new_coltag(soup, "80%", "20%"))
soup.html.unwrap()
soup.body.unwrap()
    
# html_body = html_body.replace('<td>fail','<td class="red">fail').replace('<td>xfail','<td class="blue">xfail').replace('<td>skip','<td class="yellow">skip')
# html_body = html_body.replace('<table border="1">', '<table style="width:100%; border-style: solid; border-width: thin; text-align: left; vertical-align: top; cellpadding: 0;"> <col width="20%"> <col width="80%">')
# html_body = html_body.replace('<td>','<td style=" cellpadding: 0; border: 0.1px solid black; text-align: left; vertical-align: top;">').replace('<th>','<th style="cellpadding: 0; border: 0.1px solid black; text-align: left; vertical-align: top;">')
#print(html_body)  
#print(html_body.format(str(soup)))
display(HTML(html_body.format(str(soup))))